In [1]:
%matplotlib notebook
from mpl_toolkits import mplot3d


import numpy as np
import matplotlib.pyplot as plt
import pybullet as p
import pybullet_data 
import time
import math
import random
import torch
import torch.nn as nn
from torch.autograd import Variable


In [2]:
physicsClient = p.connect(p.GUI)

In [3]:
p.setGravity(0,0,-10) 
p.resetSimulation() 
p.setAdditionalSearchPath(pybullet_data.getDataPath()) 
planeId = p.loadURDF("plane.urdf") 
robotId = p.loadURDF("iiwa7.urdf",flags=9, useFixedBase=1)

robotStartPos = [0,0,0]
robotStartOrientation = p.getQuaternionFromEuler([0,0,0])

p.resetBasePositionAndOrientation(robotId,robotStartPos,robotStartOrientation)

p.setJointMotorControlArray(robotId,range(7),p.VELOCITY_CONTROL,forces=np.zeros(7))

In [4]:
def simulate_system(x, u):
    x_next=[]
    for i in range(7):
        p.resetJointState(robotId,i,x[i],targetVelocity = x[i+7])
    
    p.setJointMotorControlArray(robotId,range(7), controlMode=p.TORQUE_CONTROL,forces=u)
    p.stepSimulation()
    for i in range(7):
        x_next.append(p.getJointStates(robotId,range(7))[i][0])
    for i in range(7):
        x_next.append(p.getJointStates(robotId,range(7))[i][1])
    x_next = np.array(x_next)
    return x_next

In [5]:
N=2000
x = np.zeros([14,N])
u= np.zeros([7, N])
x_new=np.zeros([14,N])

In [6]:
for i in range(N):
    for j in range(7):
        a=random.randint(-180,180)#angule range(-pi,pi)
        b=random.uniform(-10, 10)#velocity range
        c=random.randint(-30,30)#torque range (-200,200)
        x[j,i]=math.radians(a/math.pi)
        x[j+7,i]=b
        u[j,i]=c
            
for i in range(N):
    x_new[:,i]=simulate_system(x[:,i], u[:,i])
        
x=x.T
u = u.T
x_train = np.append(x,u,axis=1)
y_train = x_new.T
x_train = x_train.astype(np.float32)
y_train=y_train.astype(np.float32)
x_train = torch.from_numpy(x_train)
y_train = torch.from_numpy(y_train)

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [8]:
class MultiLinearRegression(nn.Module):
    def __init__(self):
        super(MultiLinearRegression, self).__init__()
        self.linear = nn.Linear(21,14)  
        
    def forward(self,x):
        out = self.linear(x)
        return out


model = MultiLinearRegression()

criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=1e-3)

epoch = 0
while epoch<=100000:
    output = model(x_train)  
    loss = criterion(output, y_train)  
    loss_value = loss.data.cpu().numpy() 
    optimizer.zero_grad()  
    loss.backward() 
    optimizer.step() 
    
    epoch += 1
    if epoch % 100 == 0: 
        print('Epoch:{}, loss:{:.6f}'.format(epoch, loss_value))
    if loss_value <= 1e-3:
        break
w = model.linear.weight.data.numpy()
b = model.linear.bias.data.numpy()

Epoch:100, loss:7.368406
Epoch:200, loss:3.390052
Epoch:300, loss:1.869053
Epoch:400, loss:1.284193
Epoch:500, loss:1.057715
Epoch:600, loss:0.968725
Epoch:700, loss:0.932599
Epoch:800, loss:0.916856
Epoch:900, loss:0.909001
Epoch:1000, loss:0.904218
Epoch:1100, loss:0.900645
Epoch:1200, loss:0.897563
Epoch:1300, loss:0.894691
Epoch:1400, loss:0.891919
Epoch:1500, loss:0.889206
Epoch:1600, loss:0.886535
Epoch:1700, loss:0.883898
Epoch:1800, loss:0.881292
Epoch:1900, loss:0.878716
Epoch:2000, loss:0.876170
Epoch:2100, loss:0.873652
Epoch:2200, loss:0.871162
Epoch:2300, loss:0.868700
Epoch:2400, loss:0.866266
Epoch:2500, loss:0.863858
Epoch:2600, loss:0.861478
Epoch:2700, loss:0.859123
Epoch:2800, loss:0.856795
Epoch:2900, loss:0.854492
Epoch:3000, loss:0.852214
Epoch:3100, loss:0.849962
Epoch:3200, loss:0.847734
Epoch:3300, loss:0.845530
Epoch:3400, loss:0.843350
Epoch:3500, loss:0.841194
Epoch:3600, loss:0.839061
Epoch:3700, loss:0.836951
Epoch:3800, loss:0.834864
Epoch:3900, loss:0.83

KeyboardInterrupt: 

In [ ]:
w = model.linear.weight.data.numpy()
b = model.linear.bias.data.numpy()
print('w:{},b:{}'.format(w,b))
print(loss)